In [ ]:
!pip install pymongo pymatgen matminer scikit-learn smact pandas atomate fireworks mp-api
!pip install pymatgen-db
!apt-get update
!apt-get install -y texlive texlive-latex-extra texlive-fonts-recommended dvipng cm-super
!pip install dask[dataframe]
!pip install dask[complete] dask-ml matminer
! pip install "pymongo[srv]"
!pip install --upgrade fireworks ruamel.yaml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### Imports ###
import json
import os
from tqdm import tqdm
from itertools import zip_longest

from pymatgen.core import Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDEntry
from pymatgen.analysis.reaction_calculator import ComputedReaction
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.io.vasp.sets import DictSet
from pymatgen.io.vasp.outputs import Vasprun

from fireworks import LaunchPad
from atomate.vasp.workflows.presets.core import wf_structure_optimization
from atomate.vasp.powerups import add_modify_incar, add_additional_fields_to_taskdocs

In [ ]:
# Import structures
with open('drive/MyDrive/data/Structures/predicted_structures.json', 'r') as f:
    tetra_element_oxide_for_calc = json.load(f)

# Convert back to pymatgen Structure objects
tetra_element_oxide_for_calc = [{'structure': Structure.from_dict(i)}
                             for i in tetra_element_oxide_for_calc]

In [ ]:
# Handles missing keys
import json
from ruamel.yaml import YAML
from fireworks import LaunchPad
from pymatgen.core import Structure
from atomate.vasp.workflows.presets.core import wf_structure_optimization
from atomate.vasp.powerups import add_modify_incar
from atomate.common.powerups import add_additional_fields_to_taskdocs

# Load the launchpad configuration from the YAML file using ruamel.yaml
yaml = YAML(typ='safe', pure=True)
with open('drive/MyDrive/data/launchpad/my_launchpad.yaml', 'r') as f:
    config = yaml.load(f)

# Initialize LaunchPad with the full configuration
launchpad = LaunchPad(
    host=config['host'],
    port=config.get('port', 27017),
    name=config['name'],
    username=config['username'],
    password=config['password'],
    authsource=config.get('authsource', 'admin'),
    mongoclient_kwargs=config.get('mongoclient_kwargs', {})
)

# WARNING: This will delete all data in the Fireworks collections in the specified database.
launchpad.reset('', require_password=False)

# Convert back to pymatgen Structure objects with error handling for missing keys
valid_structures = []
for i in tetra_element_oxide_for_calc:
    try:
        struct = Structure.from_dict(i)
        valid_structures.append({'structure': struct})
    except KeyError as e:
        print(f"Skipping entry due to missing key: {e}")

# Add unique identifiers
for n, i in enumerate(valid_structures):
    i['solaroxide_id'] = f'solaroxide_candidate_{n}'

# Add each structure to the launchpad as a modified workflow
for i in valid_structures:
    struct = i['structure']
    orig_wf = wf_structure_optimization(struct)

    # Modify INCAR parameters with Python native types
    modify_incar_params = {
        'incar_update': {
            'EDIFFG': -0.1,
            'ALGO': 'ALL',
            'EDIFF': 1e-6,
            'ENCUT': 600,
            'KPAR': 3,
            'NCORE': 24
        }
    }
    modified_wf = add_modify_incar(orig_wf, modify_incar_params=modify_incar_params)

    # Add additional fields to task documents
    update_dict = {'solaroxide_id': i['solaroxide_id']}
    modified_wf = add_additional_fields_to_taskdocs(modified_wf, update_dict=update_dict)

    # Add the modified workflow to the LaunchPad
    launchpad.add_wf(modified_wf)

print("Workflows have been added to the LaunchPad.")

DEBUG:launchpad:RESTARTED fw_id, launch_id to (1, 1)


2024-06-04 23:04:26,800 INFO Performing db tune-up


INFO:launchpad:Performing db tune-up
DEBUG:launchpad:Updating indices...


2024-06-04 23:04:29,278 INFO LaunchPad was RESET.


INFO:launchpad:LaunchPad was RESET.


Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping entry due to missing key: 'lattice'
Skipping e

In [ ]:
from mp_api.client import MPRester
import pandas as pd
from itertools import zip_longest
from pymatgen.analysis.phase_diagram import PDEntry, PhaseDiagram
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Element

api_key = 'XXXX'

# Custom grouper function
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

# Initialize the MPRester
with MPRester(api_key) as mpr:
    # Fetch provenance documents with necessary fields
    docs = mpr.materials.provenance.search(fields=["material_id", "database_IDs"])

# Extract material IDs
material_ids = [doc.material_id for doc in docs if hasattr(doc, 'database_IDs') and 'icsd' in doc.database_IDs]

# Initialize empty list to store the results
MP_competing_phases = []

# Group task IDs into chunks of 1000
chunk_size = 1000
mpid_groups = grouper(material_ids, chunk_size)

# Fields to be included in the summary
fields = ["material_id", "structure", "energy_per_atom"]

# Initialize the MPRester for querying summary data
with MPRester(api_key) as mpr:
    for group in mpid_groups:
        # Filter out None values from the current group
        mpid_list = [mpid for mpid in group if mpid is not None]

        if mpid_list:
            # Query the MP database for summary data including energy_per_atom
            search_results = mpr.materials.summary.search(material_ids=mpid_list, fields=fields)

            # Append the search results to MP_competing_phases
            MP_competing_phases.extend(search_results)

# Rename some attributes
all_competing_phases = [{'structure': i.structure, 'id': i.material_id, 'energy_atom': i.energy_per_atom} for i in MP_competing_phases]

# Convert the list of dictionaries to a DataFrame
df_all_competing_phases = pd.DataFrame(all_competing_phases)

# Display the DataFrame head
print(df_all_competing_phases.head())

Retrieving ProvenanceDoc documents:   0%|          | 0/154387 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/993 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/974 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/992 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/989 [00:00<?, ?it/s]

...

Retrieving SummaryDoc documents:   0%|          | 0/997 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/995 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/993 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/913 [00:00<?, ?it/s]

                                           structure          id  energy_atom
0  [[-1.17141631  6.11929435  0.        ] Sr, [5....    mp-19858   -14.909271
1  [[2.67260487 5.47530927 2.93280392] N, [5.4753...   mp-672234    -4.440546
2  [[0.37159337 0.37159337 0.37159337] N, [2.5359...      mp-154    -9.128751
3  [[ 1.57198517 -0.3181399  -1.57198517] N, [ 1....   mp-999498    -7.660694
4  [[0. 0. 0.] Sb, [ 2.8446115  3.2178    -0.4665...  mp-1236935   -25.438421


In [ ]:
# Add chemsys and tot_energy keys to tetra_element_oxide_for_calc
for i in tetra_element_oxide_for_calc:
    i['chemsys'] = set([el.symbol for el in i['structure'].composition.element_composition.elements])
    # Dummy, large energy as we don't know it yet
    i['tot_energy'] = 50.0

# Add chemsys and tot_energy keys to all_competing_phases
for struc in all_competing_phases:
    struc['chemsys'] = set([el.symbol for el in struc['structure'].composition.element_composition.elements])
    struc['tot_energy'] = struc['structure'].composition.num_atoms * struc['energy_atom']

# Print results for verification
print("Tetra element oxides:")
for i in tetra_element_oxide_for_calc[:5]:  # Print first five entries
    print(i)

print("\nCompeting phases:")
for struc in all_competing_phases[:5]:  # Print first five entries
    print(struc)

Tetra element oxides:
{'structure': Structure Summary
Lattice
    abc : 4.75945771 6.85033466 9.89676895
 angles : 90.0 97.21814840000002 90.00000000000001
 volume : 320.11584927188966
      A : 4.721738780752444 0.0 -0.5980138623952356
      B : -4.194620207228586e-16 6.85033466 4.194620207228586e-16
      C : 0.0 0.0 9.89676895
    pbc : True True True
PeriodicSite: Mg2+ (4.174, 1.713, 5.777) [0.8841, 0.25, 0.6371]
PeriodicSite: Mg2+ (0.5473, 5.138, 3.522) [0.1159, 0.75, 0.3629]
PeriodicSite: Fe2+ (0.6898, 5.138, 8.074) [0.1461, 0.75, 0.8247]
PeriodicSite: Fe2+ (4.032, 1.713, 1.224) [0.8539, 0.25, 0.1753]
PeriodicSite: S6+ (1.656, 1.713, 8.986) [0.3507, 0.25, 0.9292]
PeriodicSite: S6+ (3.066, 5.138, 0.3127) [0.6493, 0.75, 0.07083]
PeriodicSite: S6+ (3.141, 5.138, 5.577) [0.6653, 0.75, 0.6037]
PeriodicSite: S6+ (1.58, 1.713, 3.722) [0.3347, 0.25, 0.3963]
PeriodicSite: O2- (3.118, 5.138, 4.118) [0.6603, 0.75, 0.456]
PeriodicSite: O2- (1.604, 1.713, 5.181) [0.3397, 0.25, 0.544]
Periodic

In [ ]:
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDEntry
from mp_api.client import MPRester

# Initialize the MPRester with your API key
api_key = 'XXXX'
mpr = MPRester(api_key)

# Define the elements for which we need reference states
elements = ["Fe", "Mg", "O", "S"]

# Fetch elemental reference data
elemental_refs = mpr.get_entries_in_chemsys(elements)

# Filter only neutral elemental reference entries
elemental_entries = {entry.composition.reduced_formula: entry for entry in elemental_refs if entry.composition.is_element}

# Print the elemental entries being added
print("Elemental Entries:")
for el, entry in elemental_entries.items():
    print(f"{el}: {entry}")

# Create phase diagram objects to get potential decomposition products
for candidate in tetra_element_oxide_for_calc:
    # Get competing phases for this system
    competition = [i for i in all_competing_phases if i['chemsys'].issubset(candidate['chemsys'])]
    entries = [PDEntry(i['structure'].composition, i['tot_energy'], name=i['id']) for i in competition]

    # Add reference entries for each element in the candidate's chemical system
    candidate_elements = set(el.symbol for el in candidate['structure'].composition.elements)
    for el in candidate_elements:
        if el in elemental_entries:
            entries.append(elemental_entries[el])

    # Print the entries being used in the phase diagram
    print(f"\nEntries for candidate {candidate['structure'].composition.reduced_formula}:")
    for entry in entries:
        print(f"{entry.composition}: {entry.energy} eV (Name: {entry.name})")

    # Add the new system in
    new_entry = PDEntry(candidate['structure'].composition, candidate['tot_energy'], name='new')
    entries.append(new_entry)

    # Make phase diagram and get decomposition products
    try:
        pd = PhaseDiagram(entries)
        decomp_prods, energy_above_hull = pd.get_decomp_and_e_above_hull(new_entry)
        decomp_ids = [entry.name for entry in decomp_prods.keys()]
        candidate['decomp_prods'] = [{'structure': i['structure'],
                                      'tot_energy': i['tot_energy'],
                                      'id': i['id']} for i in competition if i['id'] in decomp_ids]
        candidate['energy_above_hull'] = energy_above_hull
    except ValueError as e:
        print(f"Error creating PhaseDiagram for {candidate['structure'].composition.reduced_formula}: {e}")
        candidate['decomp_prods'] = []  # Ensure key exists even on error

# Print results for verification
for candidate in tetra_element_oxide_for_calc:
    print(candidate)

Retrieving ThermoDoc documents:   0%|          | 0/450 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
    abc : 5.15930278 8.31687984 8.21090335
 angles : 106.15859375 90.0 90.00000000000001
 volume : 338.40558038330204
      A : 5.15930278 0.0 3.1591618176795203e-16
      B : -4.891436458555784e-16 7.9883219585620795 -2.3145631465469525
      C : 0.0 0.0 8.21090335
    pbc : True True True
PeriodicSite: Fe2+ (4.235, 6.98, 0.9871) [0.8209, 0.8738, 0.3665]
PeriodicSite: Fe2+ (1.702, 5.728, 5.464) [0.3299, 0.717, 0.8675]
PeriodicSite: Fe2+ (3.457, 1.734, 6.621) [0.6701, 0.217, 0.8675]
PeriodicSite: Fe2+ (0.9238, 2.986, 2.144) [0.1791, 0.3738, 0.3665]
PeriodicSite: Si4+ (0.8927, 2.698, 5.3) [0.173, 0.3378, 0.7406]
PeriodicSite: Si4+ (1.676, 0.002293, 2.021) [0.3249, 0.000287, 0.2463]
PeriodicSite: Si4+ (4.267, 6.692, 4.142) [0.827, 0.8378, 0.7406]
PeriodicSite: Si4+ (3.483, 3.996, 0.8641) [0.6751, 0.5003, 0.2463]
PeriodicSite: Ni2+ (3.458, 2.214, 3.484) [0.6702, 0.2771, 0.5024]
PeriodicSite: Ni2+ (4.351, 4.631, 6.633) [0.8433, 0.5797, 0.9

In [ ]:
# Define the query function
def query_db(criteria, properties):
    return list(launchpad.db.tasks.find(criteria, {prop: 1 for prop in properties}))

# Get candidate structure info
candidate_strucs = query_db({'solaroxide_id': {'$regex': 'solaroxide_candidate'}},
                            ["formula_pretty", "task_id", "output.structure",
                             "solaroxide_id", "output.energy", "output.energy_per_atom"])

# Get decomposition products info
decomp_prod_set_DFT = query_db({'solaroxide_id': {'$regex': 'Solaroxide_decompprod_'}},
                               ["formula_pretty", "task_id", "output.structure",
                                "solaroxide_id", "output.energy", "output.energy_per_atom"])

In [ ]:
# Insert calculated total energy values into the original list
for i in tetra_element_oxide_for_calc:
    for e in candidate_strucs:
        if e['solaroxide_id'] == i['solaroxide_id']:
            i['tot_energy'] = e['output']['energy']

# Update energies for decomposition products
for i in tetra_element_oxide_for_calc:
    for prod in i['decomp_prods']:
        form = prod['structure'].composition.reduced_formula
        for e in decomp_prod_set_DFT:
            if e['formula_pretty'] == form:
                prod['tot_energy'] = e['output']['energy']

In [ ]:
# Work out what set of decomposition products we need to calculate energies for
decomp_prod_ids = []
for i in tetra_element_oxide_for_calc:
    print(i['structure'].composition.reduced_formula)
    if 'decomp_prods' in i:
        for prod in i['decomp_prods']:
            print("{0:10}   {1:10}   {2}".format(prod['structure'].composition.reduced_formula,
                                                 prod['id'], prod['tot_energy']))
            decomp_prod_ids.append(prod['id'])
    else:
        print("No decomposition products found for this candidate.")
decomp_prod_ids = set(decomp_prod_ids)

# Submit calcs in the same way as for candidate compounds
decomp_prod_set = []
for n, i in enumerate(decomp_prod_ids):
    for j in all_competing_phases:
        if i == j['id']:
            struc = j['structure']
    decomp_prod_set.append({'solaroxide_id': 'Solaroxide_decompprod_{}'.format(n),
                            'MP_id': i, 'structure': struc })

for i in decomp_prod_set:
    struc = i['structure']
    orig_wf = wf_structure_optimization(struc)
    modified_wf = add_modify_incar(orig_wf, modify_incar_params={'incar_update': {'EDIFFG': -0.1, 'ALGO': 'ALL',
                                                            'EDIFF': 0.000001, 'ENCUT': 600,
                                                            'KPAR': 3, 'NCORE': 24}})
    modified_wf = add_additional_fields_to_taskdocs(modified_wf,
                                                   update_dict={'solaroxide_id': i['solaroxide_id']})
    launchpad.add_wf(modified_wf)

MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
MgFe(SO4)2
FeCu(SO4)2
FeCu(SO4)2
MnSiSO6
MnSiSO6
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
TiFeSi2O7
MnSi2SO6
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
NaCaFeO3
TiMnS2O7
TiMnS2O7
CaFe2SiO6
CaFe2SiO6
CaFe2SiO6
CaFe2SiO6
CaFe2SiO6
CaFe2SiO6
CaFe2SiO6
Na2FeAgO3
Na2FeAgO3
Na2FeAgO3
Na2FeAgO3
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
FeSiNiO4
MgMn(FeO3)2
MgMn(FeO3)2
MgMn(FeO3)2
MgMn(FeO3)2
MgMnFeO4
MgMnFeO4
MgMnFeO4
MgMnFeO4
Mn2Fe(SiO4)2
Na2MnCuO3
Ca2MnSO6
Ca2MnSO6
NaMnFe2O5
NaMnFe2O5
MgMn2SiO5
MgMn2SiO5
Mn2CdSiO7
NaCuNiO2
NaCuNiO2
C

In [ ]:
# Ensure we have all the decomposition products for each candidate
def fetch_decomposition_products(candidate):
    decomp_ids = [prod['id'] for prod in candidate['decomp_prods']]
    additional_decomp_entries = query_db({'material_id': {'$in': decomp_ids}},
                                         ["material_id", "structure", "energy_per_atom"])
    for entry in additional_decomp_entries:
        candidate['decomp_prods'].append({
            'structure': entry['structure'],
            'tot_energy': entry['structure'].composition.num_atoms * entry['energy_per_atom'],
            'id': entry['material_id']
        })

for candidate in tetra_element_oxide_for_calc:
    if 'decomp_prods' not in candidate or not candidate['decomp_prods']:
        fetch_decomposition_products(candidate)

# Verify that all candidates have their decomposition products
for candidate in tetra_element_oxide_for_calc:
    print(f"Candidate: {candidate['structure'].composition.reduced_formula}")
    if 'decomp_prods' in candidate:
        for prod in candidate['decomp_prods']:
            print(f"  Decomposition Product: {prod['structure'].composition.reduced_formula} | Energy: {prod['tot_energy']} eV")

Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: MgFe(SO4)2
Candidate: FeCu(SO4)2
Candidate: FeCu(SO4)2
Candidate: MnSiSO6
Candidate: MnSiSO6
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: TiFeSi2O7
Candidate: MnSi2SO6
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: NaCaFeO3
Candidate: TiMnS2O7
Candidate: TiMnS2O7
Candidate: CaFe2SiO6
Candidate: CaFe2SiO6
Candidate: CaFe2SiO6
Candidate: CaFe2Si

In [ ]:
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.analysis.reaction_calculator import ComputedReaction, ReactionError

# Calculate ehull values
stable_count = 0
metastable_count = 0
unstable_count = 0
ehull_values = []

for struc in tetra_element_oxide_for_calc:
    comp = struc['structure'].composition.reduced_formula
    print(f"\nCandidate Composition: {comp}")
    candidate_entry = ComputedEntry(struc['structure'].composition, struc['tot_energy'])
    spacegroup = struc['structure'].get_space_group_info()

    decomp_entries = []
    for decomp_prod in struc['decomp_prods']:
        comp_decomp = decomp_prod['structure'].composition
        print(f"Decomposition Product: {comp_decomp}")
        entry = ComputedEntry(comp_decomp, decomp_prod['tot_energy'])
        decomp_entries.append(entry)

    # Print all compositions involved
    print(f"Candidate Entry: {candidate_entry.composition}")
    print("Decomposition Entries:")
    for entry in decomp_entries:
        print(entry.composition)

    # Check if total composition matches
    total_decomp_composition = sum([entry.composition for entry in decomp_entries], Composition())
    print(f"Total Decomposition Composition: {total_decomp_composition}")
    print(f"Is balanced: {candidate_entry.composition == total_decomp_composition}")

    try:
        # Use the pymatgen reaction tool
        reaction = ComputedReaction([candidate_entry], decomp_entries)
        ehull = -reaction.calculated_reaction_energy / reaction.all_comp[0].num_atoms
        struc['ehull_DFT'] = ehull
        struc['spacegroup'] = spacegroup[0]

        print("Reaction deltaE {0:7.2f}  |   {1}  | e_hull/atom  {2:.3f}".format(reaction.calculated_reaction_energy, reaction, ehull))
        print('------------------------- -----------------------------------')
        print("Candidate: {0:8}  num_atoms (POSCAR): {1}  Energy (OUTCAR): {2:.2f}".format(candidate_entry.composition.reduced_formula, int(candidate_entry.composition.num_atoms), candidate_entry.energy))
        for decomp_entry in decomp_entries:
            print("Product:   {0:8}  num_atoms (POSCAR): {1:2}  Energy (OUTCAR): {2:.2f}".format(decomp_entry.composition.reduced_formula, int(decomp_entry.composition.num_atoms), decomp_entry.energy))
        print("=============================================================")
        print(' ')

        if ehull < 0:
            stable_count += 1
        elif ehull > 0.1:
            unstable_count += 1
        else:
            metastable_count += 1

    except ReactionError as e:
        print(f"Error balancing reaction for {struc['structure'].composition.reduced_formula}: {e}")

print ('*******************************')
print("Total Stable:     {0:3}    e_hull = 0 eV".format(stable_count))
print("Total metastable: {0:3}    0 < e_hull < 0.1 eV".format(metastable_count))
print("Total unstable:   {0:3}    e_hull > 0.1 eV".format(unstable_count))
print ('*******************************')

In [ ]:
from pymatgen.io.vasp.sets import DictSet
import os

# Identify metastable and stable compositions
meta_and_stable = [i for i in tetra_element_oxide_for_calc if i['ehull_DFT'] <= 0.1]

# Insert relaxed structure into metastable results
for i in meta_and_stable:
    for e in candidate_strucs:
        if e['solaroxide_id'] == i['solaroxide_id']:
            i['relaxed_structure'] = Structure.from_dict(e['output']['structure'])

# Define the calculation parameters
calc_config = {'INCAR': {'ALGO': 'ALL', 'GGA': 'PE', 'EDIFF': 1e-06, 'EDIFFG': -0.01, 'ENMAX': 550,
                  'ISMEAR': 0, 'ISPIN': 2, 'LORBIT': 11, 'LREAL': 'AUTO', 'NELM': 100,
                  'NSW': 0, 'PREC': 'Accurate', 'SIGMA': 0.1, 'NCORE': 24, 'KPAR': 8,
                  'LHFCALC': '.TRUE.', 'PRECFOCK': 'fast', 'AEXX': 0.25,
                  'TIME': 0.30, 'HFSCREEN': 0.207,
                 'MAGMOM': {'Ce': 5, 'Ce3+': 1, 'Co': 5, 'Co3+': 0.6, 'Co4+': 1, 'Cr': 5,
                            'Dy3+': 5, 'Er3+': 3, 'Eu': 10, 'Eu2+': 7, 'Eu3+': 6, 'Fe': 5,
                            'Gd3+': 7, 'Ho3+': 4, 'La3+': 0.6, 'Lu3+': 0.6, 'Mn': 5, 'Mn3+': 4,
                            'Mn4+': 3, 'Mo': 5, 'Nd3+': 3, 'Ni': 5, 'Pm3+': 4, 'Pr3+': 2, 'Sm3+': 5,
                            'Tb3+': 6, 'Tm3+': 2, 'V': 5, 'W': 5, 'Yb3+': 1}},
        'KPOINTS': {'reciprocal_density': 64},
        'POTCAR': {'Ac': 'Ac', 'Ag': 'Ag', 'Al': 'Al', 'Ar': 'Ar', 'As': 'As', 'Au': 'Au', 'B': 'B',
                   'Ba': 'Ba_sv', 'Be': 'Be_sv', 'Bi': 'Bi', 'Br': 'Br', 'C': 'C', 'Ca': 'Ca_sv',
                   'Cd': 'Cd', 'Ce': 'Ce', 'Cl': 'Cl', 'Co': 'Co', 'Cr': 'Cr_pv', 'Cs': 'Cs_sv',
                   'Cu': 'Cu_pv', 'Dy': 'Dy_3', 'Er': 'Er_3', 'Eu': 'Eu', 'F': 'F', 'Fe': 'Fe_pv',
                   'Ga': 'Ga_d', 'Gd': 'Gd', 'Ge': 'Ge_d', 'H': 'H', 'He': 'He', 'Hf': 'Hf_pv',
                   'Hg': 'Hg', 'Ho': 'Ho_3', 'I': 'I', 'In': 'In_d', 'Ir': 'Ir', 'K': 'K_sv',
                   'Kr': 'Kr', 'La': 'La', 'Li': 'Li_sv', 'Lu': 'Lu_3', 'Mg': 'Mg_pv', 'Mn':
                   'Mn_pv', 'Mo': 'Mo_pv', 'N': 'N', 'Na': 'Na_pv', 'Nb': 'Nb_pv', 'Nd': 'Nd_3',
                   'Ne': 'Ne', 'Ni': 'Ni_pv', 'Np': 'Np', 'O': 'O', 'Os': 'Os_pv', 'P': 'P',
                   'Pa': 'Pa', 'Pb': 'Pb_d', 'Pd': 'Pd', 'Pm': 'Pm_3', 'Pr': 'Pr_3', 'Pt': 'Pt',
                   'Pu': 'Pu', 'Rb': 'Rb_sv', 'Re': 'Re_pv', 'Rh': 'Rh_pv', 'Ru': 'Ru_pv', 'S': 'S',
                   'Sb': 'Sb', 'Sc': 'Sc_sv', 'Se': 'Se', 'Si': 'Si', 'Sm': 'Sm_3', 'Sn': 'Sn_d',
                   'Sr': 'Sr_sv', 'Ta': 'Ta_pv', 'Tb': 'Tb_3', 'Tc': 'Tc_pv', 'Te': 'Te', 'Th': 'Th',
                   'Ti': 'Ti_pv', 'Tl': 'Tl_d', 'Tm': 'Tm_3', 'U': 'U', 'V': 'V_pv', 'W': 'W_pv',
                   'Xe': 'Xe', 'Y': 'Y_sv', 'Yb': 'Yb_2', 'Zn': 'Zn', 'Zr': 'Zr_sv'}}

# Distribute the necessary files
for n, i in enumerate(meta_and_stable):
    dirname = './BG_CALCS/{0}'.format(i['solaroxide_id'])
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    v  = DictSet(i['relaxed_structure'], config_dict = calc_config)
    v.write_input(dirname)

In [ ]:
from pymatgen.io.vasp import Vasprun

# Bandgap analysis
directory = 'drive/MyDrive/data/BG_CALCS'
print(directory,".....")
for root, subdirs, files in os.walk(directory):
    if subdirs:
        for subdir in subdirs:
            try:
                vasprun = Vasprun(os.path.join(directory, subdir, 'vasprun.xml'))
                print(subdir, vasprun.as_dict()['formula_pretty'],
                      vasprun.converged_electronic, vasprun.converged_ionic,
                      vasprun.eigenvalue_band_properties[0])
            except Exception as e:
                print(f'XXXX NOPE for {subdir}: {e}')

drive/MyDrive/data/BG_CALCS .....
